In [1]:
import os
import subprocess
import commands
import time
from tqdm import tqdm_notebook.tqdm

def generate_sms(mat, dic):
    items = sorted(dic.items())
    result_string = ""
    for key, val in items:
        if val != 0:
            result_string += "{} {} {}\n".format(key[0]+1, key[1]+1, val)
    result_string += "0 0 0\n"
    create_sms_file(mat, result_string)

def create_sms_file(mat, result_string):
    sms_file= open("spasm/bench/bivariant_matrix.sms","w+")
    sms_file.write('{} {} M\n'.format(mat.nrows(), mat.ncols()))
    sms_file.write(result_string)
    return sms_file

def run_spasm_rank():    
    spasm_script = 'cat spasm/bench/bivariant_matrix.sms | ./spasm/bench/rank_hybrid'
    response = commands.getstatusoutput(spasm_script)
    rank = response[1].split("s rank = ",1)[1]
    return rank

In [4]:
n = 0
p = 0
result_matrix = matrix
enumeration_dictionary = {}
result_dictionary = {}
result_array = []
eqn_row = 0

# Returns matrix / dictionary generated by relations
def generate_matrix(n_val, p_val):
    #sets global variables
    global n, p, result_matrix, result_dictionary, eqn_row
    n = n_val
    p = p_val
    eqn_row = 0
    generate_enumerated_permutation()
    result_matrix = generate_initial_matrix()
    result_dictionary = generate_initial_dictionary()
    
    populate_matrix()
    return result_matrix, result_dictionary


# Identifies permutation    
# def enumerate_permutation(perm):
#     sorted_perm=sorted(perm)
#     size=len(perm)
#     c=sum(binomial(p-1+i,i+1) for i in range(size))
#     return c-sum(binomial(p-2+size-sorted_perm[i]-i,size-i) for i in range(size))

def generate_enumerated_permutation():
    timerValue = time.time()


    global enumeration_dictionary
#     c=sum(binomial(p-1+i,i+1) for i in range(n))
#     c-sum(binomial(p-2+n-a[i]-i,n-i) for i in range(n)) (set tuple value to this, may be unnecessary)
    count = 0
    for aset in Subsets(p-1+n,n):
        alist=list(aset)
        alist.sort()
        a=[alist[i]-i-1 for i in range(n)]
        enumeration_dictionary[tuple(a)] = count
        count += 1
    print("Populating enumerated list took {} seconds".format(time.time() % timerValue))
        
def enumerate_permutation(perm):
    return enumeration_dictionary[tuple(sorted(perm))]


# Generates initial values for the Matrix
def generate_initial_matrix():
    nvars=binomial(p-1+n,n)
    neqs=1+sum(binomial(p-1+k,k)*binomial(p-1+n-k,n-k) for k in [2]) 
    M=matrix(QQ,neqs,nvars,sparse=True)
    eqn=0;j=enumerate_permutation([0 for i in [1..n]]);M[eqn,j]=1
    return M

# Generates initial values for the Dictionary
def generate_initial_dictionary():
    return {(0,0):1}

# Sets column values
def calculate_column_values(a, b, k):
    for i in range(k):
        a_generated = [(a[i1]-a[i])%p for i1 in range(k)]
        a_generated[i] = a[i]
        j=enumerate_permutation(a_generated+b)

        #matrix
#         result_matrix[eqn_row, j]=result_matrix[eqn_row,j]-1/a.count(a[i])
        
        #dictionary
        if (eqn_row, j) in result_dictionary:
            result_dictionary[(eqn_row,j)] = result_dictionary[(eqn_row, j)]-1/a.count(a[i])
        else:
            result_dictionary[(eqn_row,j)] = -1/a.count(a[i])
            
# Iterates through all permutations of a, b
def iterate_permutations(k, handle_permutation):
    global eqn_row
    for aset in tqdm_notebook(Subsets(p-1+k,k)): 
#     for aset in Subsets(p-1+k,k): 
        alist=list(aset)
        alist.sort()
        a=[alist[i]-i-1 for i in range(k)]
        for bset in Subsets(p-1+n-k,n-k):
            eqn_row=eqn_row+1
            blist=list(bset)
            blist.sort()
            b=[blist[i]-i-1 for i in range(n-k)]
            handle_permutation(a, b, k)
# Handles what to do with each iterated permutation
def evaluate_permutation(a, b, k):
    j=enumerate_permutation(a+b)
    result_matrix[eqn_row,j]=1
    result_dictionary[(eqn_row, j)] = 1
    calculate_column_values(a, b, k)

# Starts process of populating matrix / dictionary
def populate_matrix(): 
    for k in [2]:
        iterate_permutations(2, evaluate_permutation)

In [ ]:
for i in [2..500]:
    generated_matrix, generated_dictionary = generate_matrix(5,i)

    #     matrix solution
#     rank = generated_matrix.rank()

    #     dictionary solution
    timerValue = time.time()
    generate_sms(generated_matrix, generated_dictionary)
    rank = run_spasm_rank()

    result = binomial(p-1+n, n)-int(rank)

    print("Computing rank took: {} seconds".format(time.time() % timerValue))
    print("{} {}".format(i, result))
    print("-------------------------")

Populating enumerated list took 0.000496864318848 seconds



Generating Matrix took 0.03671002388 seconds
Computing rank + generating matrix took: 0.547646045685 seconds
2 0
-------------------------
Populating enumerated list took 0.00113797187805 seconds



Generating Matrix took 0.0340349674225 seconds
Computing rank + generating matrix took: 0.0613491535187 seconds
3 0
-------------------------
Populating enumerated list took 0.00201511383057 seconds



Generating Matrix took 0.0410308837891 seconds
Computing rank + generating matrix took: 0.0700578689575 seconds
4 0
-------------------------
Populating enumerated list took 0.00432705879211 seconds



Generating Matrix took 0.050409078598 seconds
Computing rank + generating matrix took: 0.0826010704041 seconds
5 0
-------------------------
Populating enumerated list took 0.00764417648315 seconds


...


Generating Matrix took 555.341372013 seconds
Computing rank + generating matrix took: 1157.57212996 seconds
40 0
-------------------------
Populating enumerated list took 35.594627142 seconds



Generating Matrix took 669.684695005 seconds
Computing rank + generating matrix took: 105846.851208 seconds
41 0
-------------------------
Populating enumerated list took 39.6843409538 seconds



Generating Matrix took 671.397241831 seconds
Computing rank + generating matrix took: 6752.58875704 seconds
42 0
-------------------------
Populating enumerated list took 47.4108388424 seconds



Generating Matrix took 839.056121826 seconds
Computing rank + generating matrix took: 2275.29985499 seconds
43 0
-------------------------
Populating enumerated list took 63.2642478943 seconds



Generating Matrix took 972.829792023 seconds
Computing rank + generating matrix took: 2629.86735797 seconds
44 0
-------------------------
Populating enumerated list took 2125.54112387 seconds
